Copyright (c) Microsoft Corporation. All rights reserved.  

Licensed under the MIT License.

In [1]:
# Import Workspace
import os
# import dotenv
from azureml.core import Workspace, Datastore
from azureml.exceptions import ProjectSystemException

# _ = dotenv.load_dotenv('.env')

# Create or Import Azure Machine Learning Workspace

In [2]:
# Fetch Workspace Name, Resource Group and Subscription ID from environment variables
# If these aren't defined as Env Vars, then replace the "None" entries below
WORKSPACE_NAME = os.environ.get('AML_WORKSPACE', None)
RESOURCE_GROUP = os.environ.get('AML_RG', None)
SUBSCRIPTION_ID = os.environ.get('AML_SUB_ID', None)
REGION_NAME = os.environ.get('AML_REGION', None)

In [3]:
try:
    ws = Workspace(subscription_id=SUBSCRIPTION_ID,
                   resource_group=RESOURCE_GROUP,
                   workspace_name=WORKSPACE_NAME)
    ws.write_config()
except ProjectSystemException:
    print("Workspace Not Found - please create workspace using code below")
    print()
    print('Workspace Name: \t', WORKSPACE_NAME)
    print('Resource Group: \t', RESOURCE_GROUP)
    print('Subscription ID: \t', SUBSCRIPTION_ID)

ValidationError: Parameter 'resource_group_name' can not be None.

In [ ]:
# # If the above code fails, uncomment this code block to create a new Workspace
# ws = Workspace.create(subscription_id=SUBSCRIPTION_ID,
#                       resource_group=RESOURCE_GROUP,
#                       name=WORKSPACE_NAME,
#                       location=REGION_NAME)
# ws.write_config()

In [4]:
ws = Workspace.from_config()

# Create CPU Cluster
We can create a CPU-based AML Managed Compute Cluster

In [5]:
from azureml.core.compute import AmlCompute

In [6]:
cpu_cluster_name = 'cpu-cluster-west'

if ws.compute_targets.get(cpu_cluster_name):
    print(f"'{cpu_cluster_name}' found. Skipping cluster creation")
    ct = ws.compute_targets[cpu_cluster_name]
else:
    print(f"'{cpu_cluster_name}' not found. Creating a new AML Manged Compute cluster.")
    
    cpu_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS13_v2',
                                                       vm_priority='dedicated',
                                                       min_nodes=0,
                                                       max_nodes=5,
                                                       idle_seconds_before_scaledown=360,
                                                       tags={'project_name': "lab-demo"},
                                                       description="An AML Managed compute cluster leveraging DS Series VMs"
                                                      )
    ct = AmlCompute.create(workspace=ws,
                      name=cpu_cluster_name, 
                      provisioning_configuration=cpu_config)
    ct.wait_for_completion(show_output=True)

'cpu-cluster' found. Skipping cluster creation


In [ ]:
# # If you would like to see the supported VM sizes, you can get a list with the supported_vmsizes method
# AmlCompute.supported_vmsizes(ws)

# Create GPU Cluster

In [ ]:
gpu_cluster_name = 'gpu-cluster'

if ws.compute_targets.get(gpu_cluster_name):
    print(f"'{gpu_cluster_name}' found. Skipping cluster creation")
    ct = ws.compute_targets[gpu_cluster_name]
else:
    print(f"'{gpu_cluster_name}' not found. Creating a new AML Manged Compute cluster.")
    
    gpu_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                       vm_priority='dedicated',
                                                       min_nodes=0,
                                                       max_nodes=5,
                                                       idle_seconds_before_scaledown=360,
                                                       tags={'project_name': "lab-demo"},
                                                       description="An AML Managed compute cluster leveraging NC Series VMs"
                                                      )
    gpu_ct = AmlCompute.create(workspace=ws,
                      name=gpu_cluster_name, 
                      provisioning_configuration=gpu_config)
    gpu_ct.wait_for_completion(show_output=True)

In [ ]:
# Verify clusters are created
ct = ws.compute_targets['cpu-cluster']
cs = ct.get_status()
cs.serialize()

## Attach Data Store

In [5]:
BLOB_CONTAINER = os.environ.get("BLOB_CONTAINER_NAME", "diabetes")
BLOB_ACCOUNT = os.environ.get("BLOB_ACCT_NAME", "publicmldataeastus")
BLOB_ACCT_KEY = os.environ.get("BLOB_ACCT_KEY", "JzbwX/fvm/bjlQCkSfKGaxhVDuDAsQ22wZNwf6ngE3l/F/ArzfwJQDuJToT899dwwtVSpgrAClFR5aZ3JSvqwQ==")

In [6]:
datastore_name = 'diabetes'

if ws.datastores.get(datastore_name):
    print(f"'{datastore_name}' datastore found. Skipping registration")
else:
    print(f"'{datastore_name}' datastore not found. Registering with Workspace")
    _ = Datastore.register_azure_blob_container(workspace=ws,
                                                datastore_name='diabetes',
                                                container_name=BLOB_CONTAINER,
                                                account_name=BLOB_ACCOUNT,
                                                account_key=BLOB_ACCT_KEY)

'diabetes' datastore not found. Registering with Workspace


In [7]:
datastore_name = 'hymenoptera'

if ws.datastores.get(datastore_name):
    print(f"'{datastore_name}' datastore found. Skipping registration")
else:
    print(f"'{datastore_name}' datastore not found. Registering with Workspace")
    _ = Datastore.register_azure_blob_container(workspace=ws,
                                                datastore_name='hymenoptera',
                                                container_name='hymenoptera',
                                                account_name=BLOB_ACCOUNT,
                                                account_key=BLOB_ACCT_KEY)

'hymenoptera' datastore not found. Registering with Workspace


In [8]:
ws.datastores

{'workspacefilestore': <azureml.data.azure_storage_datastore.AzureFileDatastore at 0x7fa14d2c65c0>,
 'workspaceblobstore': <azureml.data.azure_storage_datastore.AzureBlobDatastore at 0x7fa14d2df438>,
 'publicmldataeastus__ameshousing': <azureml.data.azure_storage_datastore.AzureBlobDatastore at 0x7fa14d2df780>,
 'diabetes': <azureml.data.azure_storage_datastore.AzureBlobDatastore at 0x7fa14d2dfac8>,
 'hymenoptera': <azureml.data.azure_storage_datastore.AzureBlobDatastore at 0x7fa14d2dfdd8>}